# Import Libraries

In [ ]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import dalex as dx

from copy import copy

{
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "matplotlib": matplotlib.__version__,
    "seaborn": sns.__version__,
    "sklearn": sklearn.__version__,
    "dalex": dx.__version__,
}

In [ ]:
df = pd.read_csv("./stackoverflow_full.csv", index_col=0)
target = "Employed"

## ML prerequisites

In [ ]:
#split your data set in 2 parts : training and testing

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.3,
    random_state=42
)

In [ ]:
# Protected attribute is 0 if a man or non binary and 0 if a woman plus the age

protected = (pd.Series(np.where(X_test["Gender"] == "Woman", '1', '0'), index=X_test.index) 
             + '_' 
             + X_test.Age)
protected_train = (pd.Series(np.where(X_train["Gender"] == "Woman", '1', '0').astype(str), index=X_train.index) 
                   + '_' 
                   + X_train.Age)

# Privileged population is men under 35 years old
privileged = '0_<35'

In [ ]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

#You can change the Decision tree hyperparameters or the classifier below

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, random_state=123))
])

In [ ]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

In [ ]:
# exp_decisiontree = dx.Explainer(clf_decisiontree, df.drop(columns=[target]), df[target], verbose=False)
exp_decisiontree = dx.Explainer(clf_decisiontree, X_test, y_test, verbose=True)

In [ ]:
exp_decisiontree.model_performance().result

In [ ]:
fairness_decisiontree = exp_decisiontree.model_fairness(protected=protected, privileged=privileged)

In [ ]:
fairness_decisiontree.fairness_check(epsilon = 0.8) # default epsilon

In [ ]:
fairness_decisiontree.plot(verbose=False)

### Strategy 1.2 Pre-processing: Remove sensitive attribute

The first thing that can come to mind is to remove sensitive variables. However, this option is considered as really naive since it can have no effect. 

**When may this method works ?**

➤ If the sensitive variable conveys the bias (mostly) on its own.

    ➤ This means, the sensitive variable is correlated with the target variable
    ➤ This also means, the sensitive variable is NOT correlated at all with other explanatory variables and any combination of other explanatory variables cannot be used as a proxi for the sensitive variable. 

Most of the time, the bias is shared accross explanatory variables. For example, a bias on gender may be present in many ones (salary, education, socio-professional category, etc.)

This transformation is not possible in the dalex module (since it not a recommended option to deal with bias). To implement it, a new model has to be trained and explained. This time the sensitive variable

#### Training

In [ ]:
# Retrain a model without sensitive variables "age" and "sex"
X_train_restricted = X_train.drop(['Gender', 'Age'], axis=1)

preprocessor_restr = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

clf_decisiontree_restr = Pipeline(steps=[
    ('preprocessor', preprocessor_restr),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

clf_decisiontree_restr.fit(X_train_restricted, y_train)

#### Algorithmic performance

In [ ]:
# Create a new dalex explainer for the model without sensitive variables
exp_decisiontree_restr = dx.Explainer(clf_decisiontree_restr, X_test, y_test, verbose=True)

exp_decisiontree_restr.model_performance().result


When comparing these performance metrics with those of the model with all variables, what can you observe?

In our case, can we use this metric to compare models?

#### Fairness performance
    

In [ ]:
# Let's see effect of removing sensitive variables on bias metrics
fairness_decisiontree_restr = exp_decisiontree_restr.model_fairness(protected=protected, privileged=privileged, 
                                                                    label='DecisionTreeClassifier_no_sensitive')

fairness_decisiontree_restr.fairness_check(epsilon = 0.8) # default epsilon

fairness_decisiontree_restr.plot()

What impact does the pre-processing startegy had on the bias?

In [ ]:
# Compare 2 (or more) fairness objects in dalex (add them as list in parameters). 
# It's also possible to choose the plot type ! 
fairness_decisiontree_restr.plot([fairness_decisiontree], type='radar')